In [1]:
import numpy as np
import time
import copy
import operator
import matplotlib.pyplot as plt
%matplotlib inline

# Introduction

Bla bla bla MDP bla bla

# 2-states MDP

* $P(s_0 | s_0, a_0) = 0.5,  r = 5$
* $P(s_1 | s_0, a_0) = 0.5,  r = 5$
* $P(s_0 | s_0, a_1) = 0$
* $P(s_1 | s_0, a_1) = 1,  r = 10$
* $P(s_1 | s_0, a_2) = 0$
* $P(s_1 | s_1, a_2) = 1,  r = -1$


* $\gamma = 0.95$

In [2]:
def create_2statesworld():

    P = np.zeros((2,2,3)) # P(s'|s,a) ... our model of the environment
    P[0,0,0] = 0.5
    P[1,0,0] = 0.5
    P[1,0,1] = 1.
    P[1,1,2] = 1.

    R = np.zeros((2,3))  # R(s,a) ... the reward funciton 
    R[0,0] = 5
    R[0,1] = 10
    R[1,2] = -1

    states = [0, 1]
    actions = [[0, 1], [2]]
    next_states = [0, 1]
    gamma = 0.95
    
    return P, R, states, actions, next_states, gamma

### Implementations

In [3]:
def policy_iteration(P, R, states, actions, next_states, gamma, epsilon=1e-4):
    
    #1. INITIALIZATION
    V_k = np.zeros((2,), dtype=np.float) # V(s) ... our value function estimate for PI
    PI = np.zeros((2), dtype=np.int)     # PI(s) ... our greedy policy
    policy_stable = False
    all_k = []
    
    while not policy_stable:
        
        # 2. POLICY EVALUATION (iterates until V_k converges) 
        k = 0
        V_kplus1 = copy.deepcopy(V_k)
        delta = epsilon + 1
        while delta > epsilon:

            delta = 0
            for s in states:
                for n in next_states:

                    # Bellman's update rule
                    a = int(PI[s])
                    V_kplus1[s] += P[n,s,a] * (R[s,a] + gamma * V_k[n])

                    # Keeps biggest difference seen so far
                    delta = np.max([delta, np.abs(V_kplus1[s] - V_k[s])])

            # Updates our current estimate
            V_k = V_kplus1
            k += 1
        all_k.append(k)

        # 3. POLICY IMPROVEMENT (greedy action selection with respect to V_k)
        Q = {0: {0: 0,   # state0, action0
                 1: 0},  # state0, action1
             1: {2: 0}}  # state1, action2
        
        policy_stable = True
        old_PI = copy.deepcopy(PI)
        
        for s in states: 
            for a in actions[s]:
                for n in next_states:
                    
                    # Policy Improvement rule
                    Q[s][a] += P[n,s,a] * (R[s,a] + gamma * V_k[n])
                    
            PI[s] = max(Q[s].items(), key=operator.itemgetter(1))[0]
                    
            if old_PI[s] != PI[s]:
                policy_stable = False
    
    return V_k, all_k, PI 

In [ ]:
def value_iteration():
    
    return V, PI

In [ ]:
def modified_policy_iteration():
    
    return V, PI

### Experiments

In [4]:
P, R, states, actions, next_states, gamma = create_2statesworld()
V_k, all_k, PI = policy_iteration(P, R, states, actions, next_states, gamma)

print("Policy found in {} iterations, where each policy evaluation lasted for k = {}".format(len(all_k), all_k))
print("V = ", V_k)
print("PI = ", PI)

Policy found in 2 iterations, where each policy evaluation lasted for k = [2, 2]
V =  [ 38.82335938  -2.95      ]
PI =  [0 2]


### Discussion

Blablabla blabla

# Gridworld MDP

(Image taken from Sutton's slides on DP)
![grid world](gridworld.PNG)

In [13]:
#######################################################################
# Copyright (C)                                                       #
# 2016 Shangtong Zhang(zhangshangtong.cpp@gmail.com)                  #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

def create_gridworld(world_size):
    """
    world_size: height and width of the squared-shape gridworld
    return
        actions: list of str, possible actions
        states: list of coordinate tuples representing all non-terminal states
        nextState: list of list of dict, index 3 times to return the next state coordinate tuple
    """

    # left, up, right, down
    actions = ['L', 'U', 'R', 'D']

    # Next
    nextState = []
    for i in range(0, WORLD_SIZE):
        nextState.append([])
        for j in range(0, WORLD_SIZE):
            # Creates a dictionnary that
            next = dict()
            if i == 0:
                next['U'] = (i, j)
            else:
                next['U'] = (i - 1, j)

            if i == WORLD_SIZE - 1:
                next['D'] = (i, j)
            else:
                next['D'] = (i + 1, j)

            if j == 0:
                next['L'] = (i, j)
            else:
                next['L'] = (i, j - 1)

            if j == WORLD_SIZE - 1:
                next['R'] = (i, j)
            else:
                next['R'] = (i, j + 1)

            nextState[i].append(next)

    states = []
    for i in range(0, WORLD_SIZE):
        for j in range(0, WORLD_SIZE):
            if (i == 0 and j == 0) or (i == WORLD_SIZE - 1 and j == WORLD_SIZE - 1):
                continue
            else:
                states.append((i, j))
                
    return actions, states, nextState

### Implementations

In [20]:
def policy_iteration(world_size, states, actions, nextState, gamma, epsilon=1e-4):

    # The reward is always -1
    R = -1
    
    #1. INITIALIZATION
    V_k = np.zeros((world_size, world_size), dtype=np.float)  # V(s) ... our value function estimate for PI
    PI = np.zeros((world_size, world_size), dtype=np.int)     # PI(s) ... our greedy policy
    policy_stable = False
    all_k = []
    idx_to_a = {0:'L', 1:'U', 2:'R', 3:'D'}
    
    while not policy_stable:
        
        # 2. POLICY EVALUATION (iterates until V_k converges) 
        k = 0
        V_kplus1 = copy.deepcopy(V_k)
        delta = epsilon + 1
        while delta > epsilon:

            delta = 0
            for i, j in states:

                # Here the next state is fully defined by the policy (there is no uncertainty on the transition)
                a = idx_to_a[PI[i,j]]
                newPosition = nextState[i][j][a]
                P = 1.

                # Bellman's update rule
                V_kplus1[i, j] += P * (R + gamma * V_k[newPosition[0], newPosition[1]])

                # Keeps biggest difference seen so far
                delta = np.max([delta, np.abs(V_kplus1[i,j] - V_k[i,j])])

            # Updates our current estimate
            V_k = V_kplus1
            k += 1
        all_k.append(k)

        # 3. POLICY IMPROVEMENT (greedy action selection with respect to V_k)
        Q = np.zeros((world_size, world_size, 4), dtype=np.float)
        
        policy_stable = True
        old_PI = copy.deepcopy(PI)
        
        for i, j in states:
            for idx in range(4): # actions
                    
                # Again the next state is fully defined by the chosen action (there is no uncertainty on the transition)
                a = idx_to_a[idx]
                newPosition = nextState[i][j][a]
                P = 1.

                # Policy Improvement rule
                Q[i,j,idx] = P * (R + gamma * V_k[newPosition[0], newPosition[1]])
                    
            PI[i,j] = np.argmax(Q[i,j,:])
                    
            if old_PI[i,j] != PI[i,j]:
                policy_stable = False
    
    return V_k, all_k, PI 

In [ ]:
def value_iteration():
    
    return V, PI

In [ ]:
def modified_policy_iteration():
    
    return V, PI

In [42]:
def print_policy(policy):
    
    #idx_to_symbol = {0:'\u25C0', 1:'\u25BC', 2:'\u25B6', 3:'\u25BC'}
    idx_to_symbol = {0:'\u2190', 1:'\u2191', 2:'\u2192', 3:'\u2193'}
    
    for i in range(policy.shape[0]):
        
        string = "|"
        for j in range(policy.shape[1]):
            
            if (i == 0 and j == 0) or (i == policy.shape[0]-1 and j == policy.shape[1]-1):
                string += '\u25A0'
            else:
                string += idx_to_symbol[policy[i, j]]
        
        string += "|"
        print(string)
    
    return

### Experiments

In [44]:
actions, states, nextState = create_gridworld(world_size=4)
V_k, all_k, PI = policy_iteration(4, states, actions, nextState, gamma, epsilon=1e-4)

print("Policy found in {} iterations, where each policy evaluation lasted for k = {}".format(len(all_k), all_k))
print("V = \n", np.round(V_k))
print("PI = ")
print_policy(PI)

Policy found in 4 iterations, where each policy evaluation lasted for k = [2, 2, 2, 2]
V = 
 [[   0.   -8.  -38. -118.]
 [  -9.  -39. -119.  -86.]
 [ -45. -122.  -77.  -14.]
 [-127.  -65.  -13.    0.]]
PI = 
|■←←←|
|↑↑↑↓|
|↑↑↓↓|
|↑→→■|


### Discussion

Bla bla bla

# Conclusion

Bl abla bla bla bla

In [ ]:
def print_policy(PI):
    
    for i in range(PI.shape[0]):
        print("|O<<<|")
        print("|^<^v|")
        print("|^>vv|")
        print("|^>>O|")
    